In [8]:
from dotenv import load_dotenv

load_dotenv()

True

In [10]:
from langchain_openai import ChatOpenAI

from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent

model = ChatOpenAI(model="gpt-4o")

# Create specialized agents

def add(a: float, b: float) -> float:
    """Add two numbers."""
    return a + b

def multiply(a: float, b: float) -> float:
    """Multiply two numbers."""
    return a * b

def web_search(query: str) -> str:
    """Search the web for information."""
    return (
        "Here are the headcounts for each of the FAANG companies in 2024:\n"
        "1. **Facebook (Meta)**: 67,317 employees.\n"
        "2. **Apple**: 164,000 employees.\n"
        "3. **Amazon**: 1,551,000 employees.\n"
        "4. **Netflix**: 14,000 employees.\n"
        "5. **Google (Alphabet)**: 181,269 employees."
    )

math_agent = create_react_agent(
    model=model,
    tools=[add, multiply],
    name="math_expert",
    prompt="You are a math expert. Always use one tool at a time."
)

research_agent = create_react_agent(
    model=model,
    tools=[web_search],
    name="research_expert",
    prompt="You are a world class researcher with access to web search. Do not do any math."
)



In [13]:
workflow = create_supervisor(
    [research_agent, math_agent],
    model=model,
    prompt=(
        "You are a team supervisor managing a research expert and a math expert. "
        "For current events, use research_agent. "
        "For math problems, use math_agent."
    )
)

In [19]:
app = workflow.compile()
result = app.invoke({
    "messages": [
        {
            "role": "user",
            "content": "which is the square root of 16?"
        }
    ]
})

In [25]:
agent_name = result["messages"][-1].name
message = result["messages"][-1].content

print(f"{agent_name}: {message}")

supervisor: The square root of 16 is 4.


### Supervisor Agent with low level agent

In [27]:
from typing import Literal


from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langgraph.graph import END, START, StateGraph, MessagesState
from langgraph.graph.state import CompiledStateGraph
from langgraph.prebuilt import ToolNode

# --- Weather tool ---
@tool
def get_weather(location: str):
    """Call to get the current weather."""
    if location.lower() in ["munich"]:
        return "It's 15 degrees Celsius and cloudy."
    else:
        return "It's 32 degrees Celsius and sunny."

# We'll create a model and bind the tool so the LLM knows it can call `get_weather`.
tools = [get_weather]
model = ChatOpenAI(model="gpt-4o-mini").bind_tools(tools)

# --- Existing agent workflow definition ---
def call_model(state: MessagesState):
    """Call the LLM with the conversation so far."""
    messages = state["messages"]
    response = model.invoke(messages)
    return {"messages": [response]}

def should_continue(state: MessagesState) -> Literal["tools", END]:
    """If there's a tool call requested, go to 'tools', else end."""
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return END

weather_workflow = StateGraph(MessagesState)

tool_node = ToolNode(tools)

weather_workflow.add_node("agent", call_model)
weather_workflow.add_node("tools", tool_node)

weather_workflow.add_edge(START, "agent")
weather_workflow.add_conditional_edges("agent", should_continue)
weather_workflow.add_edge("tools", "agent")

weather_agent_graph =  weather_workflow.compile(name="weather_agent")

In [28]:
from langgraph_supervisor import create_supervisor

supervisor_workflow = create_supervisor(
    agents=[weather_agent_graph],
    model=model,
    prompt=(
        "You are a supervisor managing a weather agent. "
        "For any weather-related question, call the 'weather_agent' to handle it."
    ),
    output_mode="last_message",
    #output_mode="full_history",
    supervisor_name="supervisor_agent",
)

supervisor_app = supervisor_workflow.compile()

In [30]:
result = supervisor_app.invoke(
    {"messages": [HumanMessage(content="Hello there, how are you?")]}
)

result["messages"][-1].content

"Hello! I'm here and ready to assist you. How can I help you today?"

In [31]:
result = supervisor_app.invoke(
    {"messages": [HumanMessage(content="How is the weather in Munich?")]}
)
result["messages"][-1].content


'The weather in Munich is currently 15 degrees Celsius and cloudy. If you have any further questions, feel free to ask!'